In [1]:
!pip install transformers

In [2]:
!pip install sentence-transformers --quiet

In [3]:
!pip install textstat --quiet

In [4]:
!pip install catboost --quiet

In [5]:
import re
import numpy as np
import pandas as pd
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
import textstat
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

### Data Preprocessing

In [6]:
df = pd.read_csv("ads_creative_text_sample.csv")
df

,text,dimensions
0,Up to\n$100 off\nroundtrip\nflights to\nIrelan...,"(160, 600)"
1,yp The Real\nур\nYellow Pages\nFind cheap\ngas...,"(300, 250)"
2,Food Navigator\nUSA\nPlant-based meat:\nBeyond...,"(300, 600)"
3,MONSTROUS\nAIRFLOW\n$20-$23\nDURAMAX\nbanks\n+...,"(300, 250)"
4,YUMMY\nCOMBS\n*\na\nNutriti\nWellne\nNOW\nSafe...,"(728, 90)"
...,...,...
995,Ochrome OS\nDevice management\nmade simple and...,"(300, 600)"
996,"Continue to Recipe\n3 Easy Steps:\n1) Click ""C...","(336, 280)"
997,WIN THIS HOUSE\nAND START\nEVERY DAY HERE\nAll...,"(970, 250)"
998,EVIDENT OLYMPUS\nSZX-AR1 Augmented\nReality Mi...,"(300, 250)"


In [7]:
def clean_ad_text(text):
    text = text.replace("\n", " ")
    text = " ".join(text.split()) 
    return text


In [8]:
df["clean_text"] = df["text"].apply(clean_ad_text)
df[["text", "clean_text"]].head(5)

,text,clean_text
0,Up to\n$100 off\nroundtrip\nflights to\nIrelan...,Up to $100 off roundtrip flights to Ireland. T...
1,yp The Real\nур\nYellow Pages\nFind cheap\ngas...,yp The Real ур Yellow Pages Find cheap gas nea...
2,Food Navigator\nUSA\nPlant-based meat:\nBeyond...,Food Navigator USA Plant-based meat: Beyond th...
3,MONSTROUS\nAIRFLOW\n$20-$23\nDURAMAX\nbanks\n+...,MONSTROUS AIRFLOW $20-$23 DURAMAX banks +27% B...
4,YUMMY\nCOMBS\n*\na\nNutriti\nWellne\nNOW\nSafe...,"YUMMY COMBS * a Nutriti Wellne NOW Safest, Mos..."


In [9]:
def capital_features(text):
    if not isinstance(text, str):
        text = ""

    words = text.split()
    num_words = len(words)
    caps_words = sum(1 for w in words if w.isupper() and len(w) > 1)
    capital_ratio = caps_words / num_words if num_words else 0
    num_exclamations = text.count("!")

    return pd.Series({
        "num_words": num_words,
        "num_caps_words": caps_words,
        "capital_ratio": min(capital_ratio, 0.6),
        "num_exclamations": num_exclamations
    })


In [10]:
cap_df = df["clean_text"].apply(capital_features)
cap_df

,num_words,num_caps_words,capital_ratio,num_exclamations
0,27.0,1.0,0.037037,0.0
1,51.0,5.0,0.098039,1.0
2,23.0,14.0,0.600000,0.0
3,27.0,11.0,0.407407,0.0
4,18.0,5.0,0.277778,2.0
...,...,...,...,...
995,22.0,1.0,0.045455,0.0
996,19.0,0.0,0.000000,1.0
997,15.0,8.0,0.533333,0.0
998,10.0,3.0,0.300000,0.0


In [11]:
df = pd.concat([df, cap_df], axis=1)

In [12]:
df.columns

Index(['text', 'dimensions', 'clean_text', 'num_words', 'num_caps_words',
       'capital_ratio', 'num_exclamations'],
      dtype='object')

In [13]:
df.head()

,text,dimensions,clean_text,num_words,num_caps_words,capital_ratio,num_exclamations
0,Up to\n$100 off\nroundtrip\nflights to\nIrelan...,"(160, 600)",Up to $100 off roundtrip flights to Ireland. T...,27.0,1.0,0.037037,0.0
1,yp The Real\nур\nYellow Pages\nFind cheap\ngas...,"(300, 250)",yp The Real ур Yellow Pages Find cheap gas nea...,51.0,5.0,0.098039,1.0
2,Food Navigator\nUSA\nPlant-based meat:\nBeyond...,"(300, 600)",Food Navigator USA Plant-based meat: Beyond th...,23.0,14.0,0.600000,0.0
3,MONSTROUS\nAIRFLOW\n$20-$23\nDURAMAX\nbanks\n+...,"(300, 250)",MONSTROUS AIRFLOW $20-$23 DURAMAX banks +27% B...,27.0,11.0,0.407407,0.0
4,YUMMY\nCOMBS\n*\na\nNutriti\nWellne\nNOW\nSafe...,"(728, 90)","YUMMY COMBS * a Nutriti Wellne NOW Safest, Mos...",18.0,5.0,0.277778,2.0


In [14]:
df.isna().sum()

text                0
dimensions          0
clean_text          0
num_words           0
num_caps_words      0
capital_ratio       0
num_exclamations    0
dtype: int64

### Cleaning of Clean_Text

In [15]:
df["clean_text"] = df["clean_text"].fillna("")

In [16]:
df["clean_text"].head(5)

0    Up to $100 off roundtrip flights to Ireland. T...
1    yp The Real ур Yellow Pages Find cheap gas nea...
2    Food Navigator USA Plant-based meat: Beyond th...
3    MONSTROUS AIRFLOW $20-$23 DURAMAX banks +27% B...
4    YUMMY COMBS * a Nutriti Wellne NOW Safest, Mos...
Name: clean_text, dtype: object

### Twitter Roberta Sentiment Model

In [17]:
sentiment_pipe = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment"
)

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

RobertaForSequenceClassification LOAD REPORT from: cardiffnlp/twitter-roberta-base-sentiment
Key                             | Status     |  | 
--------------------------------+------------+--+-
roberta.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [18]:
sentiment_pipe("LIMITED TIME OFFER! BUY ONE GET ONE FREE")

[{'label': 'LABEL_2', 'score': 0.7480954527854919}]

In [19]:
sentiment_pipe("Buy one Get one free")

[{'label': 'LABEL_1', 'score': 0.6027004718780518}]

In [20]:
def sentiment_score(text):
    if not isinstance(text, str) or text.strip() == "":
        return 0.5  # neutral fallback

    result = sentiment_pipe(text)[0]["label"]

    mapping = {
        "LABEL_2": 1.0,  # positive
        "LABEL_1": 0.5,  # neutral
        "LABEL_0": 0.0   # negative
    }

    return mapping[result]


In [21]:
df["sentiment_score_test"] = df["clean_text"].head(10).apply(sentiment_score)

In [22]:
df[["clean_text", "sentiment_score_test"]].head(10)

,clean_text,sentiment_score_test
0,Up to $100 off roundtrip flights to Ireland. T...,0.5
1,yp The Real ур Yellow Pages Find cheap gas nea...,0.5
2,Food Navigator USA Plant-based meat: Beyond th...,0.5
3,MONSTROUS AIRFLOW $20-$23 DURAMAX banks +27% B...,0.5
4,"YUMMY COMBS * a Nutriti Wellne NOW Safest, Mos...",1.0
5,ROSELINLIN FREE SHIPPING 10,0.5
6,MENARDS GIFT CARD MENARDS GIFT CARDS MEWARDS G...,0.5
7,"From Hello to Help, 211 is Here 2.1.1 Get Conn...",0.5
8,ALL ABOUT CIRCUITS Need a Battery Refresher? 9...,0.5
9,UNITED MileagePlus. Get ready and bon voyage U...,0.5


In [23]:
df[["clean_text", "sentiment_score_test"]].head(10)

,clean_text,sentiment_score_test
0,Up to $100 off roundtrip flights to Ireland. T...,0.5
1,yp The Real ур Yellow Pages Find cheap gas nea...,0.5
2,Food Navigator USA Plant-based meat: Beyond th...,0.5
3,MONSTROUS AIRFLOW $20-$23 DURAMAX banks +27% B...,0.5
4,"YUMMY COMBS * a Nutriti Wellne NOW Safest, Mos...",1.0
5,ROSELINLIN FREE SHIPPING 10,0.5
6,MENARDS GIFT CARD MENARDS GIFT CARDS MEWARDS G...,0.5
7,"From Hello to Help, 211 is Here 2.1.1 Get Conn...",0.5
8,ALL ABOUT CIRCUITS Need a Battery Refresher? 9...,0.5
9,UNITED MileagePlus. Get ready and bon voyage U...,0.5


In [24]:
df["sentiment_score"] = df["clean_text"].apply(sentiment_score)

In [25]:
df["sentiment_score"].value_counts()

sentiment_score
0.5    729
1.0    252
0.0     19
Name: count, dtype: int64

In [26]:
df[["clean_text", "sentiment_score"]].sample(10)

,clean_text,sentiment_score
279,HashiCorp Improve IT Ops productivity up to 75...,1.0
373,Thousands of teens in foster care can't wait t...,1.0
160,We Stand Together,1.0
472,INDULGE YOUR HOUSE WITH THE TREATMENT SHE DESE...,0.5
83,CHATTANOOGA DENTAL CARE SITION AND CENTRAL ENT...,0.5
881,Have you been to Boswell's lately? BOSWELL'S H...,0.5
433,IRWIN View All Special Offers at: www.irwinzon...,0.5
423,IS YOUR COVERAGE UP-TO-DATE? • Home • Auto • F...,0.5
711,H she 100% Bios College of Education D Belin-B...,0.5
268,"BioGaia. There's good bacteria, and then there...",1.0


In [27]:
df.head(20)

,text,dimensions,clean_text,num_words,num_caps_words,capital_ratio,num_exclamations,sentiment_score_test,sentiment_score
0,Up to\n$100 off\nroundtrip\nflights to\nIrelan...,"(160, 600)",Up to $100 off roundtrip flights to Ireland. T...,27.0,1.0,0.037037,0.0,0.5,0.5
1,yp The Real\nур\nYellow Pages\nFind cheap\ngas...,"(300, 250)",yp The Real ур Yellow Pages Find cheap gas nea...,51.0,5.0,0.098039,1.0,0.5,0.5
2,Food Navigator\nUSA\nPlant-based meat:\nBeyond...,"(300, 600)",Food Navigator USA Plant-based meat: Beyond th...,23.0,14.0,0.600000,0.0,0.5,0.5
3,MONSTROUS\nAIRFLOW\n$20-$23\nDURAMAX\nbanks\n+...,"(300, 250)",MONSTROUS AIRFLOW $20-$23 DURAMAX banks +27% B...,27.0,11.0,0.407407,0.0,0.5,0.5
4,YUMMY\nCOMBS\n*\na\nNutriti\nWellne\nNOW\nSafe...,"(728, 90)","YUMMY COMBS * a Nutriti Wellne NOW Safest, Mos...",18.0,5.0,0.277778,2.0,1.0,1.0
5,ROSELINLIN\nFREE SHIPPING\n10,"(970, 250)",ROSELINLIN FREE SHIPPING 10,4.0,3.0,0.600000,0.0,0.5,0.5
6,MENARDS\nGIFT CARD\nMENARDS\nGIFT CARDS\nMEWAR...,"(300, 250)",MENARDS GIFT CARD MENARDS GIFT CARDS MEWARDS G...,19.0,19.0,0.600000,1.0,0.5,0.5
7,"From Hello\nto Help,\n211 is Here\n2.1.1\nGet ...","(300, 250)","From Hello to Help, 211 is Here 2.1.1 Get Conn...",14.0,2.0,0.142857,0.0,0.5,0.5
8,ALL ABOUT\nCIRCUITS\nNeed a Battery\nRefresher...,"(300, 250)",ALL ABOUT CIRCUITS Need a Battery Refresher? 9...,23.0,13.0,0.565217,1.0,0.5,0.5
9,UNITED\nMileagePlus.\nGet ready and bon voyage...,"(300, 250)",UNITED MileagePlus. Get ready and bon voyage U...,21.0,1.0,0.047619,0.0,0.5,0.5


In [28]:
df.to_csv("ads_with_sentiment.csv", index=False)

In [29]:
df.head()

,text,dimensions,clean_text,num_words,num_caps_words,capital_ratio,num_exclamations,sentiment_score_test,sentiment_score
0,Up to\n$100 off\nroundtrip\nflights to\nIrelan...,"(160, 600)",Up to $100 off roundtrip flights to Ireland. T...,27.0,1.0,0.037037,0.0,0.5,0.5
1,yp The Real\nур\nYellow Pages\nFind cheap\ngas...,"(300, 250)",yp The Real ур Yellow Pages Find cheap gas nea...,51.0,5.0,0.098039,1.0,0.5,0.5
2,Food Navigator\nUSA\nPlant-based meat:\nBeyond...,"(300, 600)",Food Navigator USA Plant-based meat: Beyond th...,23.0,14.0,0.600000,0.0,0.5,0.5
3,MONSTROUS\nAIRFLOW\n$20-$23\nDURAMAX\nbanks\n+...,"(300, 250)",MONSTROUS AIRFLOW $20-$23 DURAMAX banks +27% B...,27.0,11.0,0.407407,0.0,0.5,0.5
4,YUMMY\nCOMBS\n*\na\nNutriti\nWellne\nNOW\nSafe...,"(728, 90)","YUMMY COMBS * a Nutriti Wellne NOW Safest, Mos...",18.0,5.0,0.277778,2.0,1.0,1.0


### Semantic Persuation Score

In [30]:
# sentence-transformers/all-MiniLM-L6-v2

In [31]:
df["clean_text"].head(5)

0    Up to $100 off roundtrip flights to Ireland. T...
1    yp The Real ур Yellow Pages Find cheap gas nea...
2    Food Navigator USA Plant-based meat: Beyond th...
3    MONSTROUS AIRFLOW $20-$23 DURAMAX banks +27% B...
4    YUMMY COMBS * a Nutriti Wellne NOW Safest, Mos...
Name: clean_text, dtype: object

In [32]:
df["clean_text"] = df["clean_text"].fillna("")

In [33]:
sbert = SentenceTransformer("all-MiniLM-L6-v2")

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [34]:
persuasion_anchors = [
    "limited time offer",
    "exclusive deal just for you",
    "win big prizes",
    "best price guaranteed",
    "don’t miss this opportunity",
    "start your journey today",
    "high quality product at discount"
]

In [35]:
anchor_embeddings = sbert.encode(
    persuasion_anchors,
    convert_to_tensor=True
)

In [36]:
### Test data frame
test_text = "LIMITED TIME OFFER! BUY NOW AND SAVE BIG"

test_emb = sbert.encode(test_text, convert_to_tensor=True)
similarity = util.cos_sim(test_emb, anchor_embeddings)

similarity

tensor([[0.7249, 0.3643, 0.2568, 0.3477, 0.1331, 0.0988, 0.2707]])

In [37]:
similarity.max().item()

0.7248973846435547

In [38]:
def persuasion_score(text):
    if not isinstance(text, str) or text.strip() == "":
        return 0.0

    emb = sbert.encode(text, convert_to_tensor=True)
    sim = util.cos_sim(emb, anchor_embeddings).max().item()

    return float(np.clip(sim, 0, 1))


In [39]:
df["persuasion_score_test"] = df["clean_text"].head(10).apply(persuasion_score)
df[["clean_text", "persuasion_score_test"]].head(10)

,clean_text,persuasion_score_test
0,Up to $100 off roundtrip flights to Ireland. T...,0.198016
1,yp The Real ур Yellow Pages Find cheap gas nea...,0.222818
2,Food Navigator USA Plant-based meat: Beyond th...,0.151504
3,MONSTROUS AIRFLOW $20-$23 DURAMAX banks +27% B...,0.262872
4,"YUMMY COMBS * a Nutriti Wellne NOW Safest, Mos...",0.172790
5,ROSELINLIN FREE SHIPPING 10,0.294787
6,MENARDS GIFT CARD MENARDS GIFT CARDS MEWARDS G...,0.187448
7,"From Hello to Help, 211 is Here 2.1.1 Get Conn...",0.173179
8,ALL ABOUT CIRCUITS Need a Battery Refresher? 9...,0.105081
9,UNITED MileagePlus. Get ready and bon voyage U...,0.336201


In [40]:
df["persuasion_score"] = df["clean_text"].apply(persuasion_score)

In [41]:
df["persuasion_score"].describe()

count    1000.000000
mean        0.217312
std         0.081536
min         0.029564
25%         0.160739
50%         0.208714
75%         0.262613
max         0.543086
Name: persuasion_score, dtype: float64

In [42]:
df.drop(columns=["persuasion_score_test"], inplace=True)

### CTA Score

In [43]:
df["clean_text"].head(5)

0    Up to $100 off roundtrip flights to Ireland. T...
1    yp The Real ур Yellow Pages Find cheap gas nea...
2    Food Navigator USA Plant-based meat: Beyond th...
3    MONSTROUS AIRFLOW $20-$23 DURAMAX banks +27% B...
4    YUMMY COMBS * a Nutriti Wellne NOW Safest, Mos...
Name: clean_text, dtype: object

In [44]:
df["clean_text"] = df["clean_text"].fillna("")

In [45]:
CTA_WORDS = [
    "buy", "shop", "order", "sign up", "signup", "register",
    "download", "click", "start", "get", "try", "apply",
    "claim", "win", "join", "subscribe", "book",

    "buy now", "shop now", "order now", "start now",
    "get started", "join now", "apply now", "book now",
    "reserve now", "enroll now",

    "free trial", "try free", "start free", "get free",
    "free access", "download free", "free preview",
    "no cost", "risk free", "cancel anytime",

    "save now", "get discount", "unlock savings",
    "best deal", "exclusive offer", "special price",
    "flat off", "price drop", "cashback available",
    "grab the deal",

    "limited time", "ends soon", "last chance",
    "act now", "hurry", "don’t miss",
    "today only", "offer expires", "final call",

    "learn more", "explore", "discover",
    "see how it works", "view details",
    "check it out", "find out", "watch demo",

    "get instant access", "claim your offer",
    "buy now and save", "join free today",
    "limited offer act now", "download now it’s free"
]


In [46]:
## Basic CTA scoring function
def cta_score(text):
    if not isinstance(text, str) or text.strip() == "":
        return 0.0

    text = text.lower()

    hits = sum(1 for word in CTA_WORDS if word in text)

    # normalize: 0 CTAs → 0, 3+ → 1.0
    return min(hits / 3, 1.0)


In [47]:
test_ads = [
    "BUY NOW and save big",
    "Sign up today to get exclusive access",
    "Food Navigator USA",
    "Click here to download now",
    "Searching for Speakers"
]

for ad in test_ads:
    print(ad, "→", cta_score(ad))


BUY NOW and save big → 1.0
Sign up today to get exclusive access → 0.6666666666666666
Food Navigator USA → 0.0
Click here to download now → 0.6666666666666666
Searching for Speakers → 0.0


In [48]:
df["cta_score_test"] = df["clean_text"].head(10).apply(cta_score)
df[["clean_text", "cta_score_test"]].head(10)

,clean_text,cta_score_test
0,Up to $100 off roundtrip flights to Ireland. T...,1.000000
1,yp The Real ур Yellow Pages Find cheap gas nea...,0.333333
2,Food Navigator USA Plant-based meat: Beyond th...,0.333333
3,MONSTROUS AIRFLOW $20-$23 DURAMAX banks +27% B...,0.000000
4,"YUMMY COMBS * a Nutriti Wellne NOW Safest, Mos...",0.666667
5,ROSELINLIN FREE SHIPPING 10,0.000000
6,MENARDS GIFT CARD MENARDS GIFT CARDS MEWARDS G...,0.666667
7,"From Hello to Help, 211 is Here 2.1.1 Get Conn...",0.333333
8,ALL ABOUT CIRCUITS Need a Battery Refresher? 9...,0.333333
9,UNITED MileagePlus. Get ready and bon voyage U...,0.666667


In [49]:
df["cta_score"] = df["clean_text"].apply(cta_score)

In [50]:
df["cta_score"].value_counts()

cta_score
0.000000    396
0.333333    356
0.666667    185
1.000000     63
Name: count, dtype: int64

In [51]:
df[
    ["clean_text", "cta_score"]
].sample(10)

,clean_text,cta_score
604,GRAB TOP TITLES FOR FREE! (UP TO $100) JUST PA...,0.666667
862,YOU WERE ONCE SONICALLY LOST. NOW YOU'VE BEEN ...,0.000000
398,"black hat ASIA 2023 MAY 9-12, 2023 MARINA BAY ...",0.333333
754,Capital One Shopping Coupon codes found! Capit...,0.666667
316,D AB CRICIIRUSH COLLECT PLAY. WIN UNLOCK EXPER...,0.666667
332,Dying Liver? Most People Notice THIS Too Late....,0.000000
641,CANTWELL MATTRESS CO. Voted #1 Mattress Store ...,0.000000
318,SUPPORT LOCAL JOURNALISM IVLRN Public Media OP...,0.000000
694,AARP AMERICANS ARE TALKING. CONGRESS MUST LIST...,0.000000
271,VISIT DR BRAGG TO EARN... BRAGGING RIGHTS % BR...,0.000000


In [52]:
df.drop(columns=["cta_score_test"], inplace=True)

In [53]:
df["clean_text"].head(5)

0    Up to $100 off roundtrip flights to Ireland. T...
1    yp The Real ур Yellow Pages Find cheap gas nea...
2    Food Navigator USA Plant-based meat: Beyond th...
3    MONSTROUS AIRFLOW $20-$23 DURAMAX banks +27% B...
4    YUMMY COMBS * a Nutriti Wellne NOW Safest, Mos...
Name: clean_text, dtype: object

In [54]:
df["clean_text"] = df["clean_text"].fillna("")

In [55]:
examples = [
    "Buy now and save big",
    "Searching for speakers",
    "The company has announced a comprehensive strategy to optimize operational efficiency"
]

for text in examples:
    print(text, "→", textstat.flesch_reading_ease(text))

Buy now and save big → 117.16000000000003
Searching for speakers → 62.79000000000002
The company has announced a comprehensive strategy to optimize operational efficiency → -19.6754545454545


### Readability Score

In [56]:
def readability_score(text):
    if not isinstance(text, str) or text.strip() == "":
        return 0.5  # neutral fallback

    score = textstat.flesch_reading_ease(text)

    return max(0, min(score / 100, 1))

In [57]:
df["readability_score_test"] = df["clean_text"].head(10).apply(readability_score)

In [58]:
df[["clean_text", "readability_score_test"]].head(10)

,clean_text,readability_score_test
0,Up to $100 off roundtrip flights to Ireland. T...,0.699938
1,yp The Real ур Yellow Pages Find cheap gas nea...,0.889050
2,Food Navigator USA Plant-based meat: Beyond th...,0.179683
3,MONSTROUS AIRFLOW $20-$23 DURAMAX banks +27% B...,0.840504
4,"YUMMY COMBS * a Nutriti Wellne NOW Safest, Mos...",0.452624
5,ROSELINLIN FREE SHIPPING 10,0.758750
6,MENARDS GIFT CARD MENARDS GIFT CARDS MEWARDS G...,0.717816
7,"From Hello to Help, 211 is Here 2.1.1 Get Conn...",0.970014
8,ALL ABOUT CIRCUITS Need a Battery Refresher? 9...,0.519229
9,UNITED MileagePlus. Get ready and bon voyage U...,0.727200


In [59]:
df["readability_score"] = df["clean_text"].apply(readability_score)

In [60]:
df["readability_score"].describe()

count    1000.000000
mean        0.593930
std         0.251079
min         0.000000
25%         0.418518
50%         0.616373
75%         0.782097
max         1.000000
Name: readability_score, dtype: float64

In [61]:
df[["clean_text", "readability_score"]].sample(10)

,clean_text,readability_score
807,MGM WINE & SPIRITS CLICK HERE TO SEE OUR SPECI...,0.725188
753,MUENCH WORKSHOPS 2017 Our focus is you. muench...,0.506650
887,"ASHLEY final touches, first impressions shop h...",0.506650
504,Take a break from the usual PLAN YOUR VISIT UP...,0.782450
421,LAMPS PLUS Let yourself shine. SHOP NOW,1.000000
34,I I Verified Coupons Available SEE CODES Capit...,0.375452
699,CREATED FOR ETERNITY SUPER TITANIUM CITIZEN ME...,0.043050
982,El YOU CAN HELP UKRAINIANS IN NEED. Even small...,0.562483
284,s FREE TAX STAMPS FROM SILENCERSHOP FOR YOUR S...,0.879973
970,HAVANA DX- ONLY $7 YOUR FIRST MONTH BONGD BRB ...,0.651682


In [62]:
df.drop(columns=["readability_score_test"], inplace=True)

## Final CTR Calculation

In [63]:
required_cols = [
    "sentiment_score",
    "capital_ratio",
    "persuasion_score",
    "cta_score",
    "readability_score"
]

missing = [c for c in required_cols if c not in df.columns]
missing

[]

In [64]:
df[required_cols] = df[required_cols].fillna(0)

In [95]:
## Proxy function
def compute_ctr_proxy(row):
    ctr = (
        0.25 * row["sentiment_score"] +
        0.20 * row["capital_ratio"] +
        0.25 * row["persuasion_score"] +
        0.15 * row["cta_score"] +
        0.15 * row["readability_score"]
    )
    return max(0, min(ctr, 1))


In [96]:
df["ctr_proxy_test"] = df.head(10).apply(compute_ctr_proxy, axis=1)

df[[
        "clean_text",
        "sentiment_score",
        "capital_ratio",
        "persuasion_score",
        "cta_score",
        "readability_score",
        "ctr_proxy_test"
]].head(10)

,clean_text,sentiment_score,capital_ratio,persuasion_score,cta_score,readability_score,ctr_proxy_test
0,Up to $100 off roundtrip flights to Ireland. T...,0.5,0.037037,0.198016,1.000000,0.699938,0.436902
1,yp The Real ур Yellow Pages Find cheap gas nea...,0.5,0.098039,0.222818,0.333333,0.889050,0.383670
2,Food Navigator USA Plant-based meat: Beyond th...,0.5,0.600000,0.151504,0.333333,0.179683,0.359828
3,MONSTROUS AIRFLOW $20-$23 DURAMAX banks +27% B...,0.5,0.407407,0.262872,0.000000,0.840504,0.398275
4,"YUMMY COMBS * a Nutriti Wellne NOW Safest, Mos...",1.0,0.277778,0.172790,0.666667,0.452624,0.516646
5,ROSELINLIN FREE SHIPPING 10,0.5,0.600000,0.294787,0.000000,0.758750,0.432509
6,MENARDS GIFT CARD MENARDS GIFT CARDS MEWARDS G...,0.5,0.600000,0.187448,0.666667,0.717816,0.499534
7,"From Hello to Help, 211 is Here 2.1.1 Get Conn...",0.5,0.142857,0.173179,0.333333,0.970014,0.392368
8,ALL ABOUT CIRCUITS Need a Battery Refresher? 9...,0.5,0.565217,0.105081,0.333333,0.519229,0.392198
9,UNITED MileagePlus. Get ready and bon voyage U...,0.5,0.047619,0.336201,0.666667,0.727200,0.427654


In [97]:
df["ctr_proxy"] = df.apply(compute_ctr_proxy, axis=1)

In [98]:
df["ctr_proxy"].describe()

count    1000.000000
mean        0.416045
std         0.102411
min         0.141852
25%         0.349670
50%         0.408393
75%         0.487074
max         0.740362
Name: ctr_proxy, dtype: float64

In [99]:
def ctr_bucket(ctr):
    if ctr >= 0.6:
        return "High"
    elif ctr >= 0.3:
        return "Medium"
    else:
        return "Low"

df["ctr_bucket"] = df["ctr_proxy"].apply(ctr_bucket)

In [100]:
df["ctr_bucket"].value_counts()

ctr_bucket
Medium    836
Low       123
High       41
Name: count, dtype: int64

In [101]:
df.drop(columns=["ctr_proxy_test"], inplace=True)

In [72]:
df

,text,dimensions,clean_text,num_words,num_caps_words,capital_ratio,num_exclamations,sentiment_score_test,sentiment_score,persuasion_score,cta_score,readability_score,ctr_proxy,ctr_bucket
0,Up to\n$100 off\nroundtrip\nflights to\nIrelan...,"(160, 600)",Up to $100 off roundtrip flights to Ireland. T...,27.0,1.0,0.037037,0.0,0.5,0.5,0.198016,1.000000,0.699938,0.429495,Medium
1,yp The Real\nур\nYellow Pages\nFind cheap\ngas...,"(300, 250)",yp The Real ур Yellow Pages Find cheap gas nea...,51.0,5.0,0.098039,1.0,0.5,0.5,0.222818,0.333333,0.889050,0.364062,Medium
2,Food Navigator\nUSA\nPlant-based meat:\nBeyond...,"(300, 600)",Food Navigator USA Plant-based meat: Beyond th...,23.0,14.0,0.600000,0.0,0.5,0.5,0.151504,0.333333,0.179683,0.239828,Low
3,MONSTROUS\nAIRFLOW\n$20-$23\nDURAMAX\nbanks\n+...,"(300, 250)",MONSTROUS AIRFLOW $20-$23 DURAMAX banks +27% B...,27.0,11.0,0.407407,0.0,0.5,0.5,0.262872,0.000000,0.840504,0.316794,Medium
4,YUMMY\nCOMBS\n*\na\nNutriti\nWellne\nNOW\nSafe...,"(728, 90)","YUMMY COMBS * a Nutriti Wellne NOW Safest, Mos...",18.0,5.0,0.277778,2.0,1.0,1.0,0.172790,0.666667,0.452624,0.461091,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Ochrome OS\nDevice management\nmade simple and...,"(300, 600)",Ochrome OS Device management made simple and s...,22.0,1.0,0.045455,0.0,NaN,1.0,0.131270,1.000000,0.724917,0.541555,Medium
996,"Continue to Recipe\n3 Easy Steps:\n1) Click ""C...","(336, 280)","Continue to Recipe 3 Easy Steps: 1) Click ""Con...",19.0,0.0,0.000000,1.0,NaN,1.0,0.089067,0.333333,0.458030,0.390971,Medium
997,WIN THIS HOUSE\nAND START\nEVERY DAY HERE\nAll...,"(970, 250)",WIN THIS HOUSE AND START EVERY DAY HERE All to...,15.0,8.0,0.533333,0.0,NaN,1.0,0.380776,0.666667,0.920625,0.583288,Medium
998,EVIDENT OLYMPUS\nSZX-AR1 Augmented\nReality Mi...,"(300, 250)",EVIDENT OLYMPUS SZX-AR1 Augmented Reality Micr...,10.0,3.0,0.300000,0.0,NaN,0.5,0.167346,0.000000,0.000000,0.166836,Low


In [102]:
df.head()

,text,dimensions,clean_text,num_words,num_caps_words,capital_ratio,num_exclamations,sentiment_score,persuasion_score,cta_score,readability_score,ctr_proxy,ctr_bucket
0,Up to\n$100 off\nroundtrip\nflights to\nIrelan...,"(160, 600)",Up to $100 off roundtrip flights to Ireland. T...,27.0,1.0,0.037037,0.0,0.5,0.198016,1.000000,0.699938,0.436902,Medium
1,yp The Real\nур\nYellow Pages\nFind cheap\ngas...,"(300, 250)",yp The Real ур Yellow Pages Find cheap gas nea...,51.0,5.0,0.098039,1.0,0.5,0.222818,0.333333,0.889050,0.383670,Medium
2,Food Navigator\nUSA\nPlant-based meat:\nBeyond...,"(300, 600)",Food Navigator USA Plant-based meat: Beyond th...,23.0,14.0,0.600000,0.0,0.5,0.151504,0.333333,0.179683,0.359828,Medium
3,MONSTROUS\nAIRFLOW\n$20-$23\nDURAMAX\nbanks\n+...,"(300, 250)",MONSTROUS AIRFLOW $20-$23 DURAMAX banks +27% B...,27.0,11.0,0.407407,0.0,0.5,0.262872,0.000000,0.840504,0.398275,Medium
4,YUMMY\nCOMBS\n*\na\nNutriti\nWellne\nNOW\nSafe...,"(728, 90)","YUMMY COMBS * a Nutriti Wellne NOW Safest, Mos...",18.0,5.0,0.277778,2.0,1.0,0.172790,0.666667,0.452624,0.516646,Medium


In [103]:
df.tail()

,text,dimensions,clean_text,num_words,num_caps_words,capital_ratio,num_exclamations,sentiment_score,persuasion_score,cta_score,readability_score,ctr_proxy,ctr_bucket
995,Ochrome OS\nDevice management\nmade simple and...,"(300, 600)",Ochrome OS Device management made simple and s...,22.0,1.0,0.045455,0.0,1.0,0.131270,1.000000,0.724917,0.550646,Medium
996,"Continue to Recipe\n3 Easy Steps:\n1) Click ""C...","(336, 280)","Continue to Recipe 3 Easy Steps: 1) Click ""Con...",19.0,0.0,0.000000,1.0,1.0,0.089067,0.333333,0.458030,0.390971,Medium
997,WIN THIS HOUSE\nAND START\nEVERY DAY HERE\nAll...,"(970, 250)",WIN THIS HOUSE AND START EVERY DAY HERE All to...,15.0,8.0,0.533333,0.0,1.0,0.380776,0.666667,0.920625,0.689954,High
998,EVIDENT OLYMPUS\nSZX-AR1 Augmented\nReality Mi...,"(300, 250)",EVIDENT OLYMPUS SZX-AR1 Augmented Reality Micr...,10.0,3.0,0.300000,0.0,0.5,0.167346,0.000000,0.000000,0.226836,Low
999,Searching for Speakers?\nCompare\nBefore You B...,"(300, 250)",Searching for Speakers? Compare Before You Buy...,13.0,2.0,0.153846,0.0,0.5,0.253527,0.333333,0.570683,0.354753,Medium


In [104]:
df["ctr_proxy"].value_counts()

ctr_proxy
0.441610    3
0.394136    2
0.384116    2
0.390971    2
0.454150    2
           ..
0.428113    1
0.381535    1
0.368323    1
0.345655    1
0.354753    1
Name: count, Length: 994, dtype: int64

## Dataset Test Final

In [105]:
df.shape

(1000, 13)

In [106]:
df.columns.tolist()

['text',
 'dimensions',
 'clean_text',
 'num_words',
 'num_caps_words',
 'capital_ratio',
 'num_exclamations',
 'sentiment_score',
 'persuasion_score',
 'cta_score',
 'readability_score',
 'ctr_proxy',
 'ctr_bucket']

In [107]:
df.dtypes

text                  object
dimensions            object
clean_text            object
num_words            float64
num_caps_words       float64
capital_ratio        float64
num_exclamations     float64
sentiment_score      float64
persuasion_score     float64
cta_score            float64
readability_score    float64
ctr_proxy            float64
ctr_bucket            object
dtype: object

In [108]:
df.isna().sum()

text                 0
dimensions           0
clean_text           0
num_words            0
num_caps_words       0
capital_ratio        0
num_exclamations     0
sentiment_score      0
persuasion_score     0
cta_score            0
readability_score    0
ctr_proxy            0
ctr_bucket           0
dtype: int64

In [109]:
df.sort_values("ctr_proxy", ascending=False)[
    ["clean_text", "ctr_proxy"]
].head(10)


,clean_text,ctr_proxy
386,BLACK FRIDAY ZOLUCKY Soft.clouds UP TO 50% OFF...,0.740362
421,LAMPS PLUS Let yourself shine. SHOP NOW,0.695817
997,WIN THIS HOUSE AND START EVERY DAY HERE All to...,0.689954
778,"HEAVY ON PRIZES. WIN UP TO $500,000! $20 PRECI...",0.688459
922,J.ME Black FRIDAY SALE 70% OFF *Free Shipping ...,0.684204
350,"Together, we game. KIOXIA FIOMA EXCERIA SSD EX...",0.681992
107,THE FASTEST & SMARTEST WAY TO SHOP FOR YOUR NE...,0.668230
685,big yachts LET US GET YOU ON THE WATER!,0.661982
550,GET MORE FROM IPAD Combo Touch - now for iPad ...,0.660988
939,★★. MEMORIAL DAY SALE Cp SAVE OVER $1600 4 PIE...,0.660280


In [111]:
df.head()

,text,dimensions,clean_text,num_words,num_caps_words,capital_ratio,num_exclamations,sentiment_score,persuasion_score,cta_score,readability_score,ctr_proxy,ctr_bucket
0,Up to\n$100 off\nroundtrip\nflights to\nIrelan...,"(160, 600)",Up to $100 off roundtrip flights to Ireland. T...,27.0,1.0,0.037037,0.0,0.5,0.198016,1.000000,0.699938,0.436902,Medium
1,yp The Real\nур\nYellow Pages\nFind cheap\ngas...,"(300, 250)",yp The Real ур Yellow Pages Find cheap gas nea...,51.0,5.0,0.098039,1.0,0.5,0.222818,0.333333,0.889050,0.383670,Medium
2,Food Navigator\nUSA\nPlant-based meat:\nBeyond...,"(300, 600)",Food Navigator USA Plant-based meat: Beyond th...,23.0,14.0,0.600000,0.0,0.5,0.151504,0.333333,0.179683,0.359828,Medium
3,MONSTROUS\nAIRFLOW\n$20-$23\nDURAMAX\nbanks\n+...,"(300, 250)",MONSTROUS AIRFLOW $20-$23 DURAMAX banks +27% B...,27.0,11.0,0.407407,0.0,0.5,0.262872,0.000000,0.840504,0.398275,Medium
4,YUMMY\nCOMBS\n*\na\nNutriti\nWellne\nNOW\nSafe...,"(728, 90)","YUMMY COMBS * a Nutriti Wellne NOW Safest, Mos...",18.0,5.0,0.277778,2.0,1.0,0.172790,0.666667,0.452624,0.516646,Medium


## The Catboost Regressor Model

In [113]:
feature_cols = [
    "sentiment_score",
    # "capital_ratio",
    "persuasion_score",
    "cta_score",
    "readability_score"
]

X = df[feature_cols]
Y = df["ctr_proxy"]

In [114]:
X.head()

,sentiment_score,persuasion_score,cta_score,readability_score
0,0.5,0.198016,1.000000,0.699938
1,0.5,0.222818,0.333333,0.889050
2,0.5,0.151504,0.333333,0.179683
3,0.5,0.262872,0.000000,0.840504
4,1.0,0.172790,0.666667,0.452624


In [115]:
Y.head()

0    0.436902
1    0.383670
2    0.359828
3    0.398275
4    0.516646
Name: ctr_proxy, dtype: float64

### train test split

In [116]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X,
    Y,
    test_size=0.2,
    random_state=42
)

In [117]:
textModel = CatBoostRegressor(
    iterations=500,
    learning_rate=0.05,
    depth=6,
    loss_function="RMSE",
    eval_metric="RMSE",
    random_seed=42,
    verbose=100
)

In [118]:
textModel.fit(
    X_train,
    Y_train,
    eval_set=(X_test, Y_test),
    use_best_model=True
)

0:	learn: 0.0974938	test: 0.1055711	best: 0.1055711 (0)	total: 1.81ms	remaining: 906ms
100:	learn: 0.0395842	test: 0.0468176	best: 0.0468176 (100)	total: 113ms	remaining: 447ms
200:	learn: 0.0364798	test: 0.0462062	best: 0.0460184 (160)	total: 228ms	remaining: 338ms
300:	learn: 0.0337684	test: 0.0463390	best: 0.0460184 (160)	total: 341ms	remaining: 225ms
400:	learn: 0.0315363	test: 0.0466534	best: 0.0460184 (160)	total: 458ms	remaining: 113ms
499:	learn: 0.0296697	test: 0.0467857	best: 0.0460184 (160)	total: 568ms	remaining: 0us

bestTest = 0.04601836714
bestIteration = 160

Shrink model to first 161 iterations.


In [119]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

Y_pred = textModel.predict(X_test)

rmse = np.sqrt(mean_squared_error(Y_test, Y_pred))
mae = mean_absolute_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)

rmse, mae, r2

(np.float64(0.04601836612669249), 0.04000149359687263, 0.8211035478452269)

In [120]:
comparison = df.loc[Y_test.index, ["clean_text"]].copy()
comparison["actual_ctr"] = Y_test
comparison["predicted_ctr"] = Y_pred

comparison.head(10)

,clean_text,actual_ctr,predicted_ctr
521,IN THE MIDST OF PLENTY THE NEW FOLK MUSICAL FR...,0.362397,0.336055
737,Young Alfred Home insurance from $35/mo Get Qu...,0.364700,0.440330
740,Anderson Family Heath-Anderson FUNERAL HOMES &...,0.209247,0.197936
660,GROCERY WORKER'S APPRECIATION FUND KJ KENDALL ...,0.312208,0.261328
411,DISCOVER BOTSWANA Africa's Most Exclusive Dest...,0.345605,0.299908
678,S NEWSLETTER THE Get The Star headline Stories...,0.352963,0.378622
626,Canva extend /* The _first_ Canva Developers e...,0.254734,0.329680
513,Take a break from 'the usual' VISIT UPTOWNCHAR...,0.273324,0.296883
859,Decanter Wine Experiences Discover great wines...,0.463879,0.515215
136,★2023★ CHOICE THE OFFICIAL AWARDS Tri State's ...,0.604714,0.544986


## Feature Importance --> Need to put this in PPT

In [121]:
### Feature importance
importance = textModel.get_feature_importance()
fi_df = pd.DataFrame({
    "feature": feature_cols,
    "importance": importance
}).sort_values(by="importance", ascending=False)

fi_df

,feature,importance
0,sentiment_score,38.744460
2,cta_score,26.361182
3,readability_score,19.987629
1,persuasion_score,14.906729


### R2 Score

In [122]:
from sklearn.metrics import r2_score

r2 = r2_score(Y_test, Y_pred)
r2

0.8211035478452269

## Save Model

In [124]:
textModel.save_model("catboost_ctr_model.cbm")